<a href="https://colab.research.google.com/github/YunhoCha/2024-AI-/blob/main/%EC%8B%A0%EB%AC%B8%EC%95%84%EC%B9%B4%EC%9D%B4%EB%B8%8C_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y wget unzip
!wget -O /content/google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i /content/google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y
!google-chrome --version
!pip install selenium
!pip install pandas

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,556 kB]
Get:14 http://security.ubuntu.co

In [2]:
import requests
import zipfile
import os

# ChromeDriver 다운로드 URL 구성
chromedriver_url = "https://storage.googleapis.com/chrome-for-testing-public/127.0.6533.119/linux64/chromedriver-linux64.zip"

# ChromeDriver 다운로드
chrome_driver_zip_path = '/content/chromedriver.zip'
with requests.get(chromedriver_url, stream=True) as r:
    with open(chrome_driver_zip_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# ChromeDriver 압축 해제
with zipfile.ZipFile(chrome_driver_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/chromedriver')

# ChromeDriver 경로 설정 및 권한 부여
chrome_driver_path = '/content/chromedriver/chromedriver-linux64/chromedriver'
os.chmod(chrome_driver_path, 0o755)

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from tqdm import tqdm

# 사용자 입력
start_id_str = input("Enter the start content ID (e.g., 00093176030): ")
end_id_str = input("Enter the end content ID (e.g., 00093176035): ")
batch_size = 50  # 한 번에 처리할 데이터 수량 설정

# 크롬 드라이버 경로 설정
chrome_driver_path = '/content/chromedriver/chromedriver-linux64/chromedriver'

# 크롬 옵션 설정
def create_chrome_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저를 화면에 띄우지 않고 백그라운드에서 실행
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1280x1696")  # 가상 디스플레이 크기 설정
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--proxy-server='direct://'")
    chrome_options.add_argument("--proxy-bypass-list=*")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-browser-side-navigation")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--disable-software-rasterizer")

    service = Service(executable_path=chrome_driver_path)  # ChromeDriver 서비스 설정
    return webdriver.Chrome(service=service, options=chrome_options)  # 설정된 옵션으로 ChromeDriver 인스턴스 생성

# 최종 데이터 저장 리스트
final_data = []

# start_id와 end_id를 정수로 변환하여 범위를 생성
start_id = int(start_id_str)
end_id = int(end_id_str)
total_count = end_id - start_id + 1  # 전체 처리할 데이터 수

try:
    # 데이터 수집을 배치로 처리
    with tqdm(total=total_count, desc="Overall Progress") as pbar:
        for batch_start_id in range(start_id, end_id + 1, batch_size):
            driver = create_chrome_driver()  # 새로운 브라우저 인스턴스를 생성
            batch_data = []  # 각 배치의 데이터를 저장할 리스트
            batch_end_id = min(batch_start_id + batch_size - 1, end_id)  # 현재 배치의 마지막 ID 계산

            try:
                for content_id in range(batch_start_id, batch_end_id + 1):
                    content_id_str = str(content_id).zfill(len(start_id_str))  # ID를 원래 길이에 맞게 0으로 채움
                    url = f"https://www.nl.go.kr/newspaper/detail.do?content_id=CNTS-{content_id_str}"

                    try:
                        driver.get(url)  # 해당 URL로 이동
                        # 특정 요소가 로드될 때까지 대기 (최대 10초)
                        WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "//td[text()='신문명']/following-sibling::td"))
                        )

                        # 데이터 추출
                        newspaper_name = driver.find_element(By.XPATH, "//td[text()='신문명']/following-sibling::td").text
                        publisher = driver.find_element(By.XPATH, "//td[text()='발행처']/following-sibling::td").text
                        issued_date = driver.find_element(By.XPATH, "//td[text()='발행일']/following-sibling::td").text
                        news_position = driver.find_element(By.XPATH, "//td[text()='기사위치']/following-sibling::td").text
                        news_type = driver.find_element(By.XPATH, "//td[text()='기사유형']/following-sibling::td").text
                        news_topic = driver.find_element(By.XPATH, "//td[text()='기사주제']/following-sibling::td").text
                        description = driver.find_element(By.XPATH, "//td[text()='주기사항']/following-sibling::td").text
                        abstract = driver.find_element(By.XPATH, "//td[text()='본문정보']/following-sibling::td").text

                        # 추출된 데이터를 배치 리스트에 추가
                        batch_data.append({
                            "Content ID": content_id_str,
                            "Newspaper Name": newspaper_name,
                            "Publisher": publisher,
                            "Issued Date": issued_date,
                            "News Position": news_position,
                            "News Type": news_type,
                            "News Topic": news_topic,
                            "Description": description,
                            "Content": abstract
                        })

                    except Exception as e:
                        print(f"Failed to retrieve data for content ID: CNTS-{content_id_str}")
                        print(e)
                        continue  # 오류 발생 시 다음 ID로 넘어감

                    pbar.update(1)  # 각 ID 처리 후 진행바 업데이트

            finally:
                driver.quit()  # 각 배치가 끝나면 브라우저를 종료

            # 배치 데이터를 최종 데이터 리스트에 추가
            final_data.extend(batch_data)

finally:
    # 모든 배치가 마치지 않았더라도, 현재까지 수집한 데이터를 엑셀 파일로 저장
    df = pd.DataFrame(final_data)
    df.to_excel('newspaper_data.xlsx', index=False)
    print("Data saved successfully.")

# DataFrame 출력
df


Enter the start content ID (e.g., 00093176030): 00093176030
Enter the end content ID (e.g., 00093176035): 00093176060


Overall Progress: 100%|██████████| 31/31 [01:10<00:00,  2.27s/it]


Data saved successfully.


,Content ID,Newspaper Name,Publisher,Issued Date,News Position,News Type,News Topic,Description,Content
0,00093176030,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 3단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",大邱愛國婦人會에셔 孤兒院設施은 已往揭載얏거니와 大抵我한婦女의 財政權能 업 程度...
1,00093176031,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 4단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",請垂詳覽本社에셔 美國시아틀大博覽會出品事件으로 廣告에 對야 僉彦의 多大 贊成을 ...
2,00093176032,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",京城本町郵便局前 篠崎シノサキ器械店平壤南門通二丁目 篠崎シノサキ支店
3,00093176033,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",本所에셔 織造 疋木으로 學生帽子를 精撰製造와 大發賣하오니 僉同胞 壹次試驗시오...
4,00093176034,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",器械好品 아니라 廉價로 發賣옵고 不完全 것이 有면 換付기 近郡僉員은 購...
5,00093176035,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 2단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",鑿井而飮고 耕田而食랴 얏더니 堯舜世界 머럿셔라 無去處之茫茫이로다出將入相야 輔...
6,00093176036,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 3단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...","壯陽復元丹 此藥專治腦痛肺弱 房事色傷 心腎虛弱 遺精多夢, 陽痿不起 病後 虛弱 不下飮食..."
7,00093176037,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 4단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",◎全國內藥商시 僉位의게 大割引大出賣化痰止咳고 順氣消滯며 水土不服에 奇效가...
8,00093176038,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 5단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",新撰測量學定價 九拾錢兪吉쥰 序金澤吉 編纂右冊은 斯學에 著名 金澤吉氏가 編纂온바 ...
9,00093176039,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 7단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",漢城鍾路慈惠藥房에셔 官許拜受 二十三種封藥과 舊藥唐材와 草材 大發賣고 且石版印刷...


In [ ]:
 # DataFrame 출력
df

,Content ID,Newspaper Name,Publisher,Issued Date,News Position,News Type,News Topic,Description,Content
0,00093176030,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 3단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",大邱愛國婦人會에셔 孤兒院設施은 已往揭載얏거니와 大抵我한婦女의 財政權能 업 程度...
1,00093176031,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,3면 4단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",請垂詳覽本社에셔 美國시아틀大博覽會出品事件으로 廣告에 對야 僉彦의 多大 贊成을 ...
2,00093176032,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",京城本町郵便局前 篠崎シノサキ器械店平壤南門通二丁目 篠崎シノサキ支店
3,00093176033,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",本所에셔 織造 疋木으로 學生帽子를 精撰製造와 大發賣하오니 僉同胞 壹次試驗시오...
4,00093176034,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 1단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",器械好品 아니라 廉價로 發賣옵고 不完全 것이 有면 換付기 近郡僉員은 購...
5,00093176035,대한매일신보(국한문) [ 大韓每日申報(국한문) ],大韓每日申報社,1909년 04월 06일,4면 2단,광고,문화,"서울: 국립중앙도서관, 2017\n발행일 : 西曆壹千九百九年四月六日 火曜日\n연호 ...",鑿井而飮고 耕田而食랴 얏더니 堯舜世界 머럿셔라 無去處之茫茫이로다出將入相야 輔...
